In [ ]:
#FEM code for non-homogeneous Dirichlet boundary condition (Paper Section 4.1)
from numpy import * 
from pylab import *
from scipy.optimize import curve_fit
import scipy.sparse  
from scipy.sparse import diags 
from scipy.integrate import odeint 
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit # to fit the data in some curve
from itertools import cycle # need in for loop to plot with different dotted lines
D = 1 #0.000366 #0.0366 # mm^2/1000minutes
#### parameter based on experimental
S_0 = 0.01 #initial penetration height
x_ref = 10 ##characterstic length x_ref
####
a_0 = 500
m_0 = 0.5
m_ref = m_0
m_D = 5
u_D = m_D/m_ref
### constraints paramter 
b = 1
#dimensionless number
A_0 = a_0*m_ref* x_ref/D

#######
h_0 = S_0/x_ref
####time #####
#deltaT = 0.001 # timestep in dimension form
t_ref = x_ref**2/D ## time scale
Tmax= 0.0001
dt_f = 5e-9
T= int(ceil(Tmax/dt_f)) 
print('T=', T)
T_dim = [round(number/(dt_f)) for number in [0,Tmax/4,Tmax/2, 3*Tmax/4]] # time slice to plot the experiment 
print('T_dim=', T_dim)
###space discretization in dimensionless form
N = 101
x = linspace(0,1,N) 
h = 1/(N -1) 
##### tridiagonal matrix for M time derivative 
M = diags([h/6, 2*h/3, h/6], [-1, 0, 1], shape=(N-1, N-1)).toarray()  
M[N-2, N-2] =  M[N-2, N-2]/2 
Min = linalg.inv(M) 
#######tridiagonal matrix for K (grad(u).grad(phi)) 
K = diags([-1/h, 2/h, -1/h], [-1, 0, 1], shape=(N-1, N-1)).toarray() 
K[N-2, N-2] =  K[N-2, N-2]/2 
#print(K) 
######## tridigonal matrix for mixed term  
diagonals = zeros((3, N-1))   # 3 diagonals 
#diagonals[0,:] = linspace(-1, -N, N) 
for i in range(1, N-2): 
    diagonals[1,i] = -h/3
diagonals[1, 0] = -h/3  
diagonals[1, N-2] = 1/6*(3*x[N-1]-h)    
for i in range(N-2):     
    diagonals[2,i+1] = x[i+2]/2 - h/3 
for i in range(N-2):     
    diagonals[0,i] = -x[i+1]/2 - h/3   
k = array([diagonals[0,0:N-2], diagonals[1,:], diagonals[2,1:]], dtype=object) 
A = diags(k,[-1,0,1]).toarray() 
#vectors e_1 and e_{N} (it is needed for boundary terms)
E1 = (N-1)*[0] 
E1[0] = 1
EN = (N-1)*[0]
EN[N-2] = 1

In = [0.5] #for linear sigma: sigma(s(t)) = 0.5*s(t)

for alpha in In:
    def Model(u, t): 
        dwdt = A_0*(u[N-2]  - alpha/x_ref*u[N-1]/m_ref) 
        dudt = (dwdt/u[N-1])*dot(dot(Min, A), u[0:N-1]) - (1/(u[N-1])**2)*dot(dot(Min, K), u[0:N-1]) - (dwdt/u[N-1])*(u[N-2])*dot(Min,EN) \
              + (-(dwdt/u[N-1])*u_D*(x[0]+2*x[1])/6 + u_D/((u[N-1])**2*h))*dot(Min,E1)    
        dudt = list(dudt) 
        dudt.append(dwdt) 
        return dudt
    u0 = (N-1)*[m_0/m_ref] 
    u0.append(h_0)
    #time discretization
    t = linspace(0, Tmax, T) 
     #solve ODE 
    z = odeint(Model, u0, t)
    # we need to add the fix value at the Dirichlet boundary 
    Dirichelt_BC = T*[u_D]
    index=0
    m=np.insert(z, index, Dirichelt_BC, axis=1)
    
    hh=m[:,-1] # extracting the data of moving boundary

    ######trasnfering back to moving doamin 
    md = np.multiply(x,np.transpose([hh])) #moving domain over time 
    mm = np.multiply(m[:,:-1],np.transpose([hh])) #need for computing the mass
    ########### back to moving space domain z coordinate by z = x*h(tau) (non-dimensional form)
    plt.figure() 
    plt.plot(md[T_dim[1], :], m[T_dim[1], :-1], linewidth=2) 
    plt.plot(md[T_dim[2], :], m[T_dim[2], :-1], linewidth=2) 
    plt.plot(md[T_dim[3], :], m[T_dim[3], :-1], linewidth=2) 
    plt.grid()
    plt.show()

    #ploting in dimensional variable
    t_1 = t*t_ref #time nodes in moving domain t = tau*t_re
    t_1 = t #for plotting in non-dimensional form
    St = x_ref*hh #z[:,N] # the position of moving boundary s(t) = x_ref*h(tau)
    St = hh #z[:,N] # the position of moving boundary s(t) = x_ref*h(tau), comment this if you want to plot in dimensional form
    plt.figure()
    plt.plot(t, hh, linewidth=2) 
    plt.grid()
    plt.show()
    
    ########### ploting in the dimensional form ######
    md_d = md*x_ref #moving lenght (domain) in dimensional form)
    md_d = md #moving lenght (domain) in non-dimensional form)
    m_ref = 1 #comment if you want to plot this in dimensional form
    plt.figure()
    plt.plot(md_d[T_dim[1],:], m_ref*m[T_dim[1],:-1], linewidth=2)
    plt.plot(md_d[T_dim[2],:], m_ref*m[T_dim[2],:-1], linewidth=2)
    plt.plot(md_d[T_dim[3],:], m_ref*m[T_dim[3],:-1], linewidth=2)
    plt.grid()
    plt.show()
    ##########
    
    plt.figure()
    plt.plot(t_1, St, linewidth=2) 
    plt.grid()
    plt.show()
    
   
    ####calculating the number of walker over time 
    Total_mass_over_time = mm[:, :].sum(axis=1)
    #print('total_walker in time=',Total_mass_over_time[: -1])
    plt.figure()
    plt.plot(t_1, h*m_ref*Total_mass_over_time[:])
    plt.xlabel(r'$\tau$')
    plt.ylabel(r'Total mass of concentration $M(\tau)$')
    plt.show()

In [ ]:
#Random walk method for non-homogeneous Dirichlet BC (Paper Section 4.1)
from math import ceil, floor, erf
from mpl_toolkits.mplot3d import Axes3D
#from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import random
import numpy as np
import time
start_time = time.time()
L=0.045 # Length of domain
#L = 0.06
########## Constants for my problem
D = 1# 0.000366 # diffusion in minutes ( mm^2/ minutes)
d = 1
beta = 0.564# mm/minutes#
H = 2.5
####  parameter based on experimental
S_0 = 0.01
x_ref = 10
####
a_0 = 500
m_0 = 0.5
m_ref = m_0
### constraints paramter 
########
T_f = 500
t_ref = x_ref**2/D
#######
b = 1
#dimensionless number
A_0 = a_0*m_ref*x_ref/D

########## 
t_max= 0.0001 # Maximum time
nn= [100, 150, 200]# Number of walkers.

K = 0.5 # this constant we define for sigma, for example, sigma(h(tau)) = 5*h(tau)
no_real = 1 # no of realization (if we want to run code for many realizatiom, now we use only 1 realization)
time_refine = 2
#Dt = [5e-08]#, 5e-08/time_refine]
dt = 5e-08/time_refine
TT = [] #store values of u for different n
SS = [] #store values of h for different n
for n in nn:    
    List_T = [] # initialize the  values of u for  n
    List_S = [] # initialize the  values of h for  n
    for j in range(no_real):
        dx = np.sqrt(2*d*dt)
        NN = int(dt/dt_f) # number to filter the data from finite element data (dt_f is defined infem)
        N_x=ceil(L/dx)
        N_t=ceil(t_max/dt) # Number of points in the spatial- and time-domains.
        T=np.zeros((N_x, N_t), dtype=int)# Matrix representing T(x,t), initially set to 0.
        s_vector=np.zeros(N_t)# Vector representing the moving boundary s(t).
        ds = np.zeros(N_t)
        ds[0] = dt*A_0*(1 - K*S_0/m_0)
        s= S_0/x_ref # initial value
        j_t=0
        j_s=floor(s/dx) # Indices for time and the position of the moving boundary s(t).
        while j_t < N_t-1 and j_s < N_x-1:
            if j_t == 0: 
                for j in range(j_s):
                    T[j, 0] = 1*n
            else: 
                T[0, j_t] = u_D*n # Non_homogeneous Dirichelt 
            s_vector[j_t] = s
            for j_x in range(N_x):
                    if T[j_x, j_t] < 0:
                        sign = -1
                    else:
                        sign = 1
                    for j in range(sign*T[j_x, j_t]):
                        p= 2*round(random.uniform(0, 1))-1 # =+-1, with P(+1)=P(-1)=1/2.
                        if j_x+p > 0 and j_x + p <= j_s  and j_x <= N_x-1: # A walker move if it has not reached the boundaries.
                            T[j_x +p, j_t+1] = T[j_x+p, j_t+1] + 1*sign
                        elif j_x + p == j_s+1: # when walker is on boundary
                            ds[j_t] = dt*A_0*(T[j_s, j_t] - (K/x_ref)*s/(m_ref)) #increment for a walker
                            P = (1/n)*np.sqrt(2)*A_0*(T[j_s, j_t]*sign - (K/x_ref)*s/(m_ref))*np.sqrt(dt) #compute P_b 
                            r = random.uniform(0,1) #uniformly generated number between 0 and 1
                            if r<P: #if r<P, walker push the boundary and satay at the same postion for next time
                                T[j_s , j_t+1] = T[j_s, j_t+1] +1
                                s = s + ds[j_t]/n*sign #when walker on boundary satisfies r<P, boundary is increased
                            else:
                                T[j_s-1, j_t+1] = T[j_s-1, j_t+1] + 1*sign 
                            j_s =   floor(s/dx)
                            
            j_t = j_t +1
        T = T/(n*no_real)
        List_T.append(T)
        List_S.append(s_vector/no_real)
    T = np.sum(List_T, 0)  
    s_vector = np.sum(List_S, 0)
    
    #ploting concentration profile in 3D
    x = np.linspace(0, (N_x-1)*dx, N_x) #space grid points
    y = np.linspace(0, (N_t-1)*dt, N_t) #time grid points
    X, Y = np.meshgrid(x, y)
    mappable = plt.cm.ScalarMappable(cmap=plt.cm.viridis)
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.plot_surface(X.transpose(), Y.transpose(), T, cmap=mappable.cmap)
    plt.colorbar(mappable)
    t_vector = y[:-1]
    print('len_t_vector=', len(t_vector))
    ax.plot3D(s_vector[0:j_t], t_vector, 0*t_vector, 'ro') 
    S_T = St[::NN]
    fig = plt.figure()
    plt.plot(t_vector, s_vector[0:j_t], 'r-')
    plt.xlabel('t')
    plt.ylabel('s(t)')
    plt.plot()
    
    ####ploting the concentration profile in non-dimensional variable 
    #print('T(x, )=', T[:, -3] )
    fig = plt.figure()
    plt.plot(x, T[:, -3], 'r-')
    plt.xlabel('z')
    plt.ylabel(r'u($\tau$, z)')
    plt.plot()
    
    ####ploting the concentration profile in dimensional variable
    fig = plt.figure(dpi=500)
    plt.plot(x, T[:, int((1/NN)*T_dim[2])], 'g-')
    plt.plot(md_d[T_dim[2], :], m[T_dim[2],:-1], linewidth=2)
    fontsize = 14
    fontweight = 'normal'
    fontproperties = {'weight' : fontweight, 'size' : fontsize}
    plt.axis([-0.00005, 0.03, 0, 11])
    plt.legend([r"RW approximation", r"FE approximation"], loc='upper right') # loc='upper left', frameon=False
    #plt.grid()
    plt.xlabel('Penetration region $z$', fontsize= fontsize, fontweight=fontweight)
    plt.ylabel(r'Concentration profile $u(\tau, z)$', fontsize= fontsize, fontweight=fontweight)
    plt.plot()
    plt.show()
    
    #ploting in dimensional or nondimensionla  form and compare the result with FE approximation
    t_d = t_vector*(x_ref**2/D)
    t_d = t_vector #comment this if you want to plot in dimenional form 
    s_vector_d = s_vector[0:j_t]*x_ref
    s_vector_d = s_vector[0:j_t] #comment this if you want to plot in dimenional form 
    plt.figure(dpi=500)
    plt.plot(t_d[:], s_vector_d[:], 'r-') # random walk solution
    plt.plot(t_d[:], S_T[:len(t_d)], 'g--') #finite element solution
    fontsize = 14
    fontweight = 'normal'
    fontproperties = {'weight' : fontweight, 'size' : fontsize}
    plt.legend([r"RW approximation", r"FE approximation"], loc='upper left') # loc='upper left', frameon=False
    plt.grid()
    plt.xlabel(r'Time', fontsize= fontsize, fontweight=fontweight)
    plt.ylabel(r'Moving front', fontsize= fontsize, fontweight=fontweight)
    plt.plot()
    
    #### ploting the concentration profile on dimesional form
    T_1 = T*m_ref
    T_1 = T ###########
    xx =  x_ref*np.linspace(0, (N_x-1)*dx, N_x) #space grid points
    yy = (t_ref)*np.linspace(0, (N_t-1)*dt, N_t) #time grid points
    XX, YY = np.meshgrid(xx, yy)
    mappable = plt.cm.ScalarMappable(cmap=plt.cm.viridis)
    mappable.set_array(T_1)
    #mappable.set_clim(0, 4) # optional
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.plot_surface(XX.transpose(), YY.transpose(), T_1, cmap=mappable.cmap, norm=mappable.norm, )
    plt.colorbar(mappable)
    t_vector_1 = yy[:-1]
    ax.plot3D(x_ref*s_vector[0:j_t], t_vector_1, 0*t_vector_1, 'ro')  
    ax.set_xlabel(r'Penetration region', rotation=150)
    ax.set_ylabel(r'Time')
    plt.show()
    
    ####calculating the number of walker over time int_u(t, x)dx 
    Total_walker = T.sum(axis=0)
    #print('total_walker in time=',dx*m_ref*Total_walker[: -1])
    
    plt.figure(dpi=500)
    plt.plot(t_d, dx*m_ref*Total_walker[: -1], 'r-')
    plt.plot(t_1, h*m_ref*Total_mass_over_time[:], 'g--')
    fontsize = 14
    fontweight = 'normal'
    fontproperties = {'weight' : fontweight, 'size' : fontsize}
    plt.legend([r"RW approximation", r"FE approximation"], loc='upper left') # loc='upper left', frameon=False
    #plt.grid()
    plt.xlabel('Time', fontsize= fontsize, fontweight=fontweight)
    plt.ylabel(r'Mass', fontsize= fontsize, fontweight=fontweight)
    plt.plot()
    #plt.show()
    TT.append(T_1)
    SS.append(s_vector_d)
    
#np.savez('mv_front_dt_25e08_n1k15002k.npz', SS) #saving the solution data 
#np.savez('con_profile_dt_25e08_n1k15002k.npz', TT) #saving the solution data 
fig = plt.figure(dpi=500)
plt.plot(t_d[:], SS[0], 'r--', linewidth = 2) # random walk solution
plt.plot(t_d[:], SS[1], 'b:', linewidth = 2) # random walk solution
plt.plot(t_d[:], SS[2], 'k-.', linewidth = 2) # random walk solution
plt.plot(t_d[:], S_T[:len(t_d)], 'g-', linewidth = 2) #finite element solution
fontsize = 15
fontweight = 'normal'
fontproperties = {'weight' : fontweight, 'size' : fontsize}
plt.axis([0, np.max(t_d[:])+0.0000005, 0, np.max(S_T[:len(t_d)])+0.001])
plt.legend([r"RWM $(n=100)$", r"RWM $(n=150)$",\
            r"RWM $(n=200)$",r"FEM"], loc='lower right', frameon=False) # loc='upper left', frameon=False
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel(r'$\tau$', fontsize= fontsize, fontweight=fontweight)
plt.ylabel(r'$h(\tau)$', fontsize= fontsize, fontweight=fontweight)
plt.show()

####ploting for differnnt values for n in same plot
fig = plt.figure(dpi=500)
plt.plot(x, TT[0][:, int((1/NN)*T_dim[2])], 'r-', linewidth = 2) #random walk solution
plt.plot(md_d[T_dim[2], :], m[T_dim[2],:-1], 'g--' ,linewidth=2) #FEM
fontsize = 15
fontweight = 'normal'
fontproperties = {'weight' : fontweight, 'size' : fontsize}
plt.legend([r"RWM (n=100)",r"FEM"], loc='upper right', frameon=False) # loc='upper left', frameon=False
#plt.grid()
plt.xlabel('$z$', fontsize= fontsize, fontweight=fontweight)
plt.ylabel(r'$u(\tau, z)$', fontsize= fontsize, fontweight=fontweight)
plt.axis([-0.00005, 0.026, -0.02, 10.4])
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.plot()
plt.show()
end_time = time.time()
print('Computation time =', (end_time-start_time)/60, 'minute(s)')
